In [ ]:
population_growth_df = population_growth_df[1:]
population_growth_df = population_growth_df.reset_index(drop=True)

In [58]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pathlib import Path

pop_2017 = Path("tx_pop_zip_2017.csv")
pop_2018 = Path("tx_pop_zip_2018.csv")
pop_2019 = Path("tx_pop_zip_2019.csv")
pop_2020 = Path("tx_pop_zip_2020.csv")
pop_2021 = Path("tx_pop_zip_2021.csv")
pop_2022 = Path("tx_pop_zip_2022.csv")

columns_delete = ['GEO_ID', 'B01003_001M', 'Unnamed: 4']
data_2017 = pd.DataFrame(pd.read_csv(pop_2017)).drop(columns = columns_delete)
data_2018 = pd.DataFrame(pd.read_csv(pop_2018)).drop(columns = columns_delete)
data_2019 = pd.DataFrame(pd.read_csv(pop_2019)).drop(columns = columns_delete)
data_2020 = pd.DataFrame(pd.read_csv(pop_2020)).drop(columns = columns_delete)
data_2021 = pd.DataFrame(pd.read_csv(pop_2021)).drop(columns = columns_delete)
data_2022 = pd.DataFrame(pd.read_csv(pop_2022)).drop(columns = columns_delete)

first_merge = pd.merge(data_2017, data_2018, on = 'NAME', how = 'outer')
first_merge = first_merge.rename(columns = {"B01003_001E_x": "A", "B01003_001E_y": "B"})
second_merge = pd.merge(first_merge, data_2019, on = 'NAME', how = 'outer')
second_merge = second_merge.rename(columns = {"B01003_001E": "C"})
third_merge = pd.merge(second_merge, data_2020, on = 'NAME', how = 'outer')
third_merge = third_merge.rename(columns = {"B01003_001E": "D"})
fourth_merge = pd.merge(third_merge, data_2021, on = 'NAME', how = 'outer')
fourth_merge = fourth_merge.rename(columns = {"B01003_001E": "E"})
population_growth_df = pd.merge(fourth_merge, data_2022, on = 'NAME', how = 'outer')
population_growth_df = final_merge.rename(columns = {"B01003_001E": "F"})
pd.set_option('display.max_rows', None)

population_growth_df.loc[0, 'NAME'] = "Zip Code"
population_growth_df.loc[0, 'A'] = "2017"
population_growth_df.loc[0, 'B'] = "2018"
population_growth_df.loc[0, 'C'] = "2019"
population_growth_df.loc[0, 'D'] = "2020"
population_growth_df.loc[0, 'E'] = "2021"
population_growth_df.loc[0, 'F'] = "2022"

population_growth_df.columns = population_growth_df.iloc[0]
population_growth_df = population_growth_df[1:]
population_growth_df["Zip Code"] = population_growth_df["Zip Code"].str.replace("ZCTA5 ", '')
population_growth_df = population_growth_df.set_index('Zip Code')
population_growth_df = population_growth_df.sort_index()
population_growth_df = population_growth_df.dropna(how="any")
population_growth_df

,2017,2018,2019,2020,2021,2022
Zip Code,,,,,,
73949,1434,1523,1635,1874,1357,1463
75001,14617,15000,14992,15418,16287,16633
75002,69688,70597,71253,73916,71102,72679
75006,50788,51136,51642,52231,48104,48062
75007,53744,54701,55500,57513,54448,54498
75009,10557,11941,14089,15348,24737,28109
75010,27822,28974,30406,30784,30129,31609
75013,38975,40675,42629,42956,44807,46649
75019,41947,42597,42888,43137,44324,44706


,2017,2018,2019,2020,2021,2022
Zip Code,,,,,,
73949,1434,1523,1635,1874,1357,1463
75001,14617,15000,14992,15418,16287,16633
75002,69688,70597,71253,73916,71102,72679
75006,50788,51136,51642,52231,48104,48062
75007,53744,54701,55500,57513,54448,54498
...,...,...,...,...,...,...
79934,25846,26289,26445,26360,26916,26620
79935,18166,17434,17850,18546,19073,18815
79936,111918,112355,111620,108668,107579,107512
